# Batch normalization

On day 13, there was a batchnorm operation between convolution (netIn) and activation in the convolution layer. We even commented it out and noticed that for our example it didn't make much difference. But it can help with training.

Batch normalization is a kind of data normalization (not dissimilar to what you did in CS 251/2). Only, instead of doing it to all the data ahead of time, we do it inside the network per batch (during training).

Batch normalization acts as a regularizer. *What other types of regularization do we know?*
* During training, we normalize the inputs by z-scoring: we subtract the mean and divide by the standard deviation, then we multiply by a scale coefficient and add an offset: $BN(x) = \gamma \odot \frac{x-\hat{\mu_B}}{\hat{\sigma_B}} + \beta$
* After training, we calculate mean and standard deviation across the entire training dataset and then fix those into the model.

You can do batchnorm inside any layer, typically after netIn and before activation; for example, in a fully connected layer or in a convolution layer. If you do it in a convolution layer you do it *across channels*.


# The Project

## netIn, Convolution Layer

So let's say you have a minibatch of 5 RGB images, each 9 x 9 in shape. The shape of that passed into netIn is:
$$5 * 3 * 9 * 9$$
(batch size $N$, channel count $D$, image dimensions $I_y$, $I_x$)

Now let's imagine we are going to pass two 2 x 2 kernels across these images. The shape of that passed into netIn is:
$$4 * 3 * 2 * 2$$
(kernel count $K$, channel count $D$, kernel size $F_y$, $F_x$)

### Padding the image

Assuming $F_y = F_x$, you first want to pad each image to $\lceil\frac{F_y-1}{2} \rceil$, giving you shape $N$, $D$, $~I_y$, $~I_x$.

### Flipping the kernel

You can *flip the kernels* around x and y to make the convolution operation more intuitive (at least some people find it so!). Think about "flattening" the image to an array, and likewise the kernel (we will come back to this later!).
* With regular non-flipped kernels, convolution of kernel $C$ at image position $i$: $\sum_{f=0}^{F_x-1}C_fI_{i+F_x/2-f}$
* With flipped kernels, convolution of kernel $C$ at image position $i$: $\sum_{f=0}^{F_x-1}C_fI_{i-F_x/2+f}$

In this flattened world, with a regular kernel one set of indices goes up while the other goes down; with the flipped kernel both march in the same direction.

### Stepping

You need nested for loops. 
1. First, for each image $I_i$, for each (flipped, if you like) kernel $C_j$, for each channel $D_k$, convolve $I_i$ with $C_j$ for $D_k$. This will give you something of shape $N$, $K*D$, $~I_y$, $~I_x$.
2. Sum the output of convolution across the color channels of the image. This will give you something of shape $N$, $K$, $~I_y$, $~I_x$.
3. Add the bias terms (one per kernel). Same shape as after 2.

*For the exact shapes given at the top of this cell, what would be the shape at the end of this layer?$

Max pooling is much easier! Let's assume one $2*2$ max pooling operation. *What will be the shape at the end of this layer?*

## Increasing efficiency

In the project (in part 3), you will use two tricks to increase code speed.

1. Use cython. Python is a very intuitive language, at the cost of efficiency. Thank goodness Stefan Behnel and others had the bright idea to enable compilation to C, which is super efficient at the cost of intuitiveness.
2. im2col for convolution. Remember a few minutes ago when we talked about "flattening" the image and kernel? 
  * With this trick, we make convolution and max pooling more efficient by flattening each kernel-size window of the (padded) image, and adding the resulting array to a huge matrix. If the input image is of shape $D, I_y, I_x$ and the kernels are of shape $K, D, F_y, F_x$ then the flattened image will be of shape $D*F_y*F_x, P$ ($P$ is influenced by stride).
  * Similarly, we flatten the kernels to get a matrix of shape $D*F_y*F_x, K$.
  * Finally, you just do a single matrix multiply of those two new matrices.
3. im2col for max pooling. Similarly, we reshape the input to max pooling so that we can then just take max across an axis.

Tricks 2 and 3 also take advantage of cython because numpy is heavily optimized using C.


# Resources

https://betterexplained.com/articles/intuitive-convolution/